# Perfil de comércio

## Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


## Inicializar a API e transferir os códigos usados

In [3]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.PERIOD_SECONDS=5


# Trade profile

Choose country and years of interest

In [4]:
country_of_interest = comtrade.m49_stome_principe
years_of_interest = comtrade.year_range(2012, 2022)   
print(comtrade.COUNTRY_CODES[country_of_interest], years_of_interest)

Sao Tome and Principe 2012,2013,2014,2015,2016,2017,2018,2019,2020,2021


### Global exports and imports

In [5]:
cols = ['reporterDesc','partnerDesc','flowDesc','primaryValueFormated']
global_trade = comtrade.get_global_stats(country_of_interest, years_of_interest)
global_trade[cols].sort_index()


/Users/jrc/develop/cipf-comtrade/comtrade.py:349: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


reporterDesc partnerDesc flowDesc  \
refYear flowCode                                               
2012    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   
2013    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   
2014    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   
2015    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   
2016    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   
2017    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   
2018    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   
2019    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   
2020    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   
2021    M         Sao Tome and Principe       World   Import   
        X         Sao Tome and Principe       World   Export   

                 primaryValueFormated  
refYear flowCode                       
2012    M              141,253,970.00  
        X                6,047,150.00  
2013    M              152,093,663.00  
        X                6,938,258.00  
2014    M              169,715,687.00  
        X               10,497,358.00  
2015    M              141,847,245.00  
        X                9,223,182.00  
2016    M              139,352,022.00  
        X               10,473,934.00  
2017    M              147,049,579.31  
        X               10,562,590.92  
2018    M              148,305,957.03  
        X               12,087,302.46  
2019    M              147,666,945.40  
        X                9,858,888.95  
2020    M              138,041,161.36  
        X               12,439,557.11  
2021    M              166,275,006.87  
        X               19,174,845.70

## Principais parceiros comerciais
### Principais destinos das exportações

In [6]:
import pandas as pd
# show more lines
pd.set_option('display.max_rows', 500)

top_partners, df = comtrade.top_partners(country_of_interest,years_of_interest,flowCode='X', 
                                    extra_cols=['partner2Desc'],
                                    rank_filter=5, return_data=True)
print(top_partners.columns)

/Users/jrc/develop/cipf-comtrade/comtrade.py:349: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


KeyError: "['rankCmd'] not in index"

In [11]:
cols=['reporterDesc','refYear','flowCode','rank','partnerDesc','partner2Desc','primaryValueFormated']
top_partners[cols].sort_values(by=['refYear','flowCode','rank'], ascending=[True,True,True]).style.format({'percent_partner': '{:,.2%}'.format})

,reporterDesc,refYear,flowCode,rank,partnerDesc,partner2Desc,primaryValueFormated
132,Sao Tome and Principe,2018,X,1,Netherlands,World,"4,563,001.25"
147,Sao Tome and Principe,2018,X,2,Portugal,World,"2,915,838.25"
47,Sao Tome and Principe,2018,X,3,Belgium,World,"2,136,397.00"
101,Sao Tome and Principe,2018,X,4,Germany,World,"1,145,170.02"
79,Sao Tome and Principe,2018,X,5,France and Monaco,World,"597,473.74"
260,Sao Tome and Principe,2019,X,1,"Areas, not elsewhere specified",World,"8,937,674.11"
236,Sao Tome and Principe,2019,X,2,Portugal,World,"173,776.10"
232,Sao Tome and Principe,2019,X,3,Netherlands,World,"166,834.23"
242,Sao Tome and Principe,2019,X,4,Slovenia,World,"161,944.84"
215,Sao Tome and Principe,2019,X,5,Belgium,World,"132,988.41"


## Principais produtos exportados

In [10]:
export_products = comtrade.top_commodities(country_of_interest,
                                               partnerCode=0,  # 0 = all partners
                                               years=years_of_interest, flowCode='X',
                                           motCode=0,rank_filter=10)


/Users/jrc/develop/cipf-comtrade/comtrade.py:349: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


In [9]:
cmd_in_country = 'perc_cmd_for_partner'
# label for the percentage of a country in the total trade of a commodity
country_in_cmd = 'perc_partner_for_cmd'
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdCode','cmdDesc', 'primaryValueFormated','perc_cmd_for_partner']
export_products[cols].style.format({cmd_in_country: '{:,.2%}'.format, country_in_cmd: '{:,.2%}'.format})

NameError: name 'export_products' is not defined

## Principais produtos exportados e destinos

In [17]:
# partnerCode=None to get all partners

#TODO isto devia ser uma variante de top_partners
export_products = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='X',
                                           motCode=0, 
                                           rank_filter=5)

In [18]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_partner','partnerDesc','rank_cmd', 'cmdCode','cmdDesc',  'primaryValueFormated','perc_cmd_for_partner','perc_partner_for_cmd']

export_products[cols].style.format({cmd_in_country: '{:,.2%}'.format, country_in_cmd: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
718,Sao Tome and Principe,2018,Export,1,Netherlands,1,18,Cocoa and cocoa preparations,"4,558,478.68",99.90%,54.26%
500,Sao Tome and Principe,2018,Export,3,Belgium,1,18,Cocoa and cocoa preparations,"2,052,457.06",96.07%,24.43%
644,Sao Tome and Principe,2018,Export,4,Germany,1,18,Cocoa and cocoa preparations,"1,084,914.51",94.74%,12.91%
608,Sao Tome and Principe,2018,Export,5,France and Monaco,1,18,Cocoa and cocoa preparations,"373,002.51",62.43%,4.44%
1064,Sao Tome and Principe,2018,Export,7,Spain,1,18,Cocoa and cocoa preparations,"106,504.03",100.00%,1.27%
866,Sao Tome and Principe,2018,Export,2,Portugal,2,90,"Optical, photographic, cinematographic, measuring, checking, medical or surgical instruments and apparatus; parts and accessories","1,287,320.43",44.15%,99.92%
637,Sao Tome and Principe,2018,Export,21,Gabon,2,90,"Optical, photographic, cinematographic, measuring, checking, medical or surgical instruments and apparatus; parts and accessories",995.83,31.42%,0.08%
433,Sao Tome and Principe,2018,Export,6,Angola,2,90,"Optical, photographic, cinematographic, measuring, checking, medical or surgical instruments and apparatus; parts and accessories",59.40,0.03%,0.00%
861,Sao Tome and Principe,2018,Export,2,Portugal,3,88,"Aircraft, spacecraft, and parts thereof","674,171.26",23.12%,95.72%
649,Sao Tome and Principe,2018,Export,4,Germany,3,88,"Aircraft, spacecraft, and parts thereof","30,127.75",2.63%,4.28%


### Principais produtos e principais parceiros

In [26]:
export_products = comtrade.top_commodities(country_of_interest,partnerCode=None,years=years_of_interest, flowCode='X',
                                           motCode=0, 
                                           partner_first=True,  # show partner first
                                           rank_filter=5)

In [27]:
export_products[cols].style.format({cmd_in_country: '{:,.2%}'.format, country_in_cmd: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
20,Sao Tome and Principe,2020,Export,1,Netherlands,1,18,Cocoa and cocoa preparations,"3,726,708.78",64.02%,55.02%
146,Sao Tome and Principe,2020,Export,1,Netherlands,2,15,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","1,958,022.96",33.63%,49.70%
101,Sao Tome and Principe,2020,Export,1,Netherlands,3,88,"Aircraft, spacecraft, and parts thereof","136,699.30",2.35%,24.93%
17,Sao Tome and Principe,2020,Export,2,Belgium,1,18,Cocoa and cocoa preparations,"1,717,590.09",85.21%,25.36%
138,Sao Tome and Principe,2020,Export,2,Belgium,2,15,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","264,934.63",13.14%,6.73%
2,Sao Tome and Principe,2020,Export,2,Belgium,10,23,"Food industries, residues and wastes thereof; prepared animal fodder","23,751.50",1.18%,100.00%
151,Sao Tome and Principe,2020,Export,2,Belgium,13,12,"Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder","4,108.93",0.20%,31.30%
176,Sao Tome and Principe,2020,Export,2,Belgium,14,94,"Furniture; bedding, mattresses, mattress supports, cushions and similar stuffed furnishings; lamps and lighting fittings, n.e.c.; illuminated signs, illuminated name-plates and the like; prefabricated buildings","1,816.94",0.09%,16.09%
133,Sao Tome and Principe,2020,Export,3,Portugal,2,15,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","1,070,458.31",82.59%,27.17%
117,Sao Tome and Principe,2020,Export,3,Portugal,1,18,Cocoa and cocoa preparations,"98,232.58",7.58%,1.45%


## Análise de simetrias nos dados

In [53]:


partnerA = country_of_interest


### Principais destinos de exportação reportados pelo país

In [64]:
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='X',
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=None,
                     period=years_of_interest,
                     motCode = None,
                     customsCode=None,
                     echo_url=False
                     )
cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(20).style.format({'perc': '{:,.2%}'.format}) 


/Users/jrc/develop/cipf-comtrade/comtrade.py:326: UserWarning: Query returned different motCodes including 0 (all), check for duplicate results when aggregating. Use motCode = -1 to remove motCode = 0, or motCode=0 to remove details
  warnings.warn("Query returned different motCodes including 0 (all), check for duplicate results when aggregating. Use motCode = -1 to remove motCode = 0, or motCode=0 to remove details")
/Users/jrc/develop/cipf-comtrade/comtrade.py:335: UserWarning: Query returned different partner2Codes including 0 (all), check for duplicate results when aggregating. Use partner2Code = -1 to remove partner2Code = 0, or partner2Code=0 to remove details
  warnings.warn("Query returned different partner2Codes including 0 (all), check for duplicate results when aggregating. Use partner2Code = -1 to remove partner2Code = 0, or partner2Code=0 to remove details")


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
2,Sao Tome and Principe,Netherlands,World,2020,Export,"5,821,431.03",5821431.034000,5.73%,Water,True,All customs procedure codes
32,Sao Tome and Principe,Netherlands,World,2020,Export,"5,821,431.03",5821431.034000,5.73%,All modes of transport,True,All customs procedure codes
29,Sao Tome and Principe,Belgium,World,2020,Export,"2,015,828.11",2015828.113000,1.98%,All modes of transport,True,All customs procedure codes
9,Sao Tome and Principe,Belgium,World,2020,Export,"1,791,746.53",1791746.532000,1.76%,Water,True,All customs procedure codes
33,Sao Tome and Principe,Portugal,World,2020,Export,"1,296,155.04",1296155.036000,1.28%,All modes of transport,True,All customs procedure codes
34,Sao Tome and Principe,Portugal,World,2020,Export,"1,254,488.02",1254488.020000,1.23%,Water,True,All customs procedure codes
21,Sao Tome and Principe,Germany,World,2020,Export,"1,066,325.11",1066325.109000,1.05%,All modes of transport,True,All customs procedure codes
1,Sao Tome and Principe,Germany,World,2020,Export,"1,065,741.86",1065741.859000,1.05%,Water,True,All customs procedure codes
41,Sao Tome and Principe,Cameroon,World,2020,Export,"665,439.12",665439.119000,0.66%,All modes of transport,True,All customs procedure codes
16,Sao Tome and Principe,Cameroon,World,2020,Export,"665,195.01",665195.013000,0.65%,Water,True,All customs procedure codes


### Principais destinos de exportação segundo o registo da importações do resto do mundo

In [68]:
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='M',
                     cmdCode='TOTAL',
                     reporterCode=None,
                     partnerCode=partnerA,
                     partner2Code=None,
                     motCode = None,
                     customsCode=None,
                     period=years_of_interest,
                     echo_url=False
                     )
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>1000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(20).style.format({'perc': '{:,.2%}'.format}) 

/Users/jrc/develop/cipf-comtrade/comtrade.py:326: UserWarning: Query returned different motCodes including 0 (all), check for duplicate results when aggregating. Use motCode = -1 to remove motCode = 0, or motCode=0 to remove details
  warnings.warn("Query returned different motCodes including 0 (all), check for duplicate results when aggregating. Use motCode = -1 to remove motCode = 0, or motCode=0 to remove details")
/Users/jrc/develop/cipf-comtrade/comtrade.py:335: UserWarning: Query returned different partner2Codes including 0 (all), check for duplicate results when aggregating. Use partner2Code = -1 to remove partner2Code = 0, or partner2Code=0 to remove details
  warnings.warn("Query returned different partner2Codes including 0 (all), check for duplicate results when aggregating. Use partner2Code = -1 to remove partner2Code = 0, or partner2Code=0 to remove details")


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
218,Singapore,Sao Tome and Principe,World,2020,Import,"7,163,342.59",7163342.591000,4.94%,All modes of transport,True,All customs procedure codes
325,Switzerland,Sao Tome and Principe,World,2020,Import,"6,165,670.57",6165670.574000,4.25%,All modes of transport,True,All customs procedure codes
339,Switzerland,Sao Tome and Principe,World,2020,Import,"6,165,670.57",6165670.574000,4.25%,All modes of transport,True,Clearance for home use
332,Switzerland,Sao Tome and Principe,World,2020,Import,"5,095,444.70",5095444.703000,3.51%,Air,True,All customs procedure codes
341,Switzerland,Sao Tome and Principe,World,2020,Import,"5,095,444.70",5095444.703000,3.51%,Air,True,Clearance for home use
181,Netherlands,Sao Tome and Principe,World,2020,Import,"1,749,209.21",1749209.211000,1.21%,All modes of transport,True,All customs procedure codes
184,Poland,Sao Tome and Principe,World,2020,Import,"1,612,603.00",1612603.000000,1.11%,All modes of transport,True,All customs procedure codes
94,France and Monaco,Sao Tome and Principe,World,2020,Import,"1,608,272.23",1608272.227000,1.11%,All modes of transport,True,All customs procedure codes
98,France and Monaco,Sao Tome and Principe,World,2020,Import,"1,608,272.23",1608272.227000,1.11%,Other,True,All customs procedure codes
120,France and Monaco,Sao Tome and Principe,"Areas, not elsewhere specified",2020,Import,"1,608,272.23",1608272.227000,1.11%,All modes of transport,True,All customs procedure codes


## Análise de simetrias nos dados com um país específico

In [47]:
partnerB = comtrade.COUNTRY_CODES_REVERSE['Switzerland']

In [48]:
flowCode = 'X'
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
total_import = df_import_rep_a['primaryValue'].sum()
df_import_rep_a['perc'] = df_import_rep_a['primaryValue']/total_import
cols = cols + 'perc'
print(f"Importações de produtos de {comtrade.COUNTRY_CODES.get(partnerA)} por {comtrade.COUNTRY_CODES.get(partnerB)} reportados por {comtrade.COUNTRY_CODES.get(partnerA)}")
df_import_rep_a[cols]

Importações de produtos de Sao Tome and Principe por Switzerland reportados por Sao Tome and Principe


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue
0,Sao Tome and Principe,Switzerland,World,2021,Export,"84,440.07",84440.07


In [49]:
# invertemos

print(years_of_interest, partnerA, partnerB)
print(f"Exportações de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} para {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerB)}")

flowCode = 'M'
df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=None,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
df_import_rep_b[cols]

2020,2021 678 757
Exportações de produtos de Switzerland para Sao Tome and Principe reportados por Switzerland


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue
0,Switzerland,Sao Tome and Principe,World,2020,Import,"6,165,670.57",6165670.574
4,Switzerland,Sao Tome and Principe,World,2021,Import,"2,329,667.08",2329667.085
